<a href="https://colab.research.google.com/github/kumar-abhishek/handson-ml2/blob/master/easy_chat_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

from google.colab import drive
import os
import numpy as np
import pandas as pd
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
data_path = "/content/drive/My Drive/chatapp"

conv_path = os.path.join(data_path, 'conversation')
word2vec_path = os.path.join(data_path, 'word2vec')

In [11]:
# source: https://github.com/shreyans29/thesemicolon/blob/master/conversation.json
# -*- coding: utf-8 -*-
import os
import json
import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities
import pickle

nltk.download('punkt')

def preprocessing():
  #os.chdir("D:\semicolon\Deep Learning\chatbot");
  #model = gensim.models.Word2Vec.load('word2vec.bin')
  model = gensim.models.Word2Vec.load(os.path.join(word2vec_path, 'word2vec.bin'))
  #path2="corpus";

  x=[]
  y=[]

  #for filename in ['conversation_train.json', 'conversation_test.json', 'conversation_valid.json', 'conversation.json']:
  for filename in os.listdir(conv_path):
    file=open(os.path.join(conv_path, filename))
    data = json.load(file)
    cor=data["conversations"]

    
    for i in range(len(cor)):
        for j in range(len(cor[i])):
            if j<len(cor[i])-1:
                x.append(cor[i][j]);
                y.append(cor[i][j+1]);

  tok_x=[]
  tok_y=[]
  for i in range(len(x)):
      tok_x.append(nltk.word_tokenize(x[i].lower()))
      tok_y.append(nltk.word_tokenize(y[i].lower()))
      
      

  sentend=np.ones((300,),dtype=np.float32) 

  vec_x=[]
  for sent in tok_x:
      sentvec = [model[w] for w in sent if w in model.wv.vocab]
      vec_x.append(sentvec)
      
  vec_y=[]
  for sent in tok_y:
      sentvec = [model[w] for w in sent if w in model.wv.vocab]
      vec_y.append(sentvec)           
      
      
  for tok_sent in vec_x:
      tok_sent[14:]=[]
      tok_sent.append(sentend)
      

  for tok_sent in vec_x:
      if len(tok_sent)<15:
          for i in range(15-len(tok_sent)):
              tok_sent.append(sentend)    
              
  for tok_sent in vec_y:
      tok_sent[14:]=[]
      tok_sent.append(sentend)
      

  for tok_sent in vec_y:
      if len(tok_sent)<15:
          for i in range(15-len(tok_sent)):
              tok_sent.append(sentend)             
              
  with open('conversation.pickle','wb') as f:
      pickle.dump([vec_x,vec_y],f)                


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
preprocessing()

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
# -*- coding: utf-8 -*-


import os
import pickle
import numpy as np
from keras.models import Sequential
import gensim
from keras.layers.recurrent import LSTM,SimpleRNN
from sklearn.model_selection import train_test_split
import theano
theano.config.optimizer="None"

def training():
  with open('conversation.pickle', 'rb') as f:
      vec_x,vec_y=pickle.load(f)    
      
  vec_x=np.array(vec_x,dtype=np.float64)
  vec_y=np.array(vec_y,dtype=np.float64)    

  x_train,x_test, y_train,y_test = train_test_split(vec_x, vec_y, test_size=0.2, random_state=1)
      
  model=Sequential()
  model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
  model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
  model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
  model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
  model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

  model.fit(x_train, y_train, nb_epoch=10,validation_data=(x_test, y_test))
  model.save('LSTM500.h5')
  model_file = drive.CreateFile({'title' : 'LSTM500.h5'})                       
  model_file.SetContentFile('LSTM500.h5')                       
  model_file.Upload()
  """
  model.fit(x_train, y_train, nb_epoch=5,validation_data=(x_test, y_test))
  model.save('LSTM1000.h5');
  model_file = drive.CreateFile({'title' : 'LSTM1000.h5'})                       
  model_file.SetContentFile('LSTM1000.h5')                       
  model_file.Upload()

  model.fit(x_train, y_train, nb_epoch=5,validation_data=(x_test, y_test))
  model.save('LSTM1500.h5');
  model_file = drive.CreateFile({'title' : 'LSTM1500.h5'})                       
  model_file.SetContentFile('LSTM1500.h5')                       
  model_file.Upload()

  model.fit(x_train, y_train, nb_epoch=5,validation_data=(x_test, y_test))
  model.save('LSTM2000.h5');
  model_file = drive.CreateFile({'title' : 'LSTM2000.h5'})                       
  model_file.SetContentFile('LSTM2000.h5')                       
  model_file.Upload()

  model.fit(x_train, y_train, nb_epoch=5,validation_data=(x_test, y_test))
  model.save('LSTM2500.h5');
  model_file = drive.CreateFile({'title' : 'LSTM2500.h5'})                       
  model_file.SetContentFile('LSTM2500.h5')                       
  model_file.Upload()

  model.fit(x_train, y_train, nb_epoch=5,validation_data=(x_test, y_test))
  model.save('LSTM3000.h5');
  model_file = drive.CreateFile({'title' : 'LSTM3000.h5'})                       
  model_file.SetContentFile('LSTM3000.h5')                       
  model_file.Upload()

  model.fit(x_train, y_train, nb_epoch=5,validation_data=(x_test, y_test))
  model.save('LSTM3500.h5');
  model_file = drive.CreateFile({'title' : 'LSTM3500.h5'})                       
  model_file.SetContentFile('LSTM3500.h5')                       
  model_file.Upload()

  model.fit(x_train, y_train, nb_epoch=5,validation_data=(x_test, y_test))
  model.save('LSTM4000.h5');
  model_file = drive.CreateFile({'title' : 'LSTM4000.h5'})                       
  model_file.SetContentFile('LSTM4000.h5')                       
  model_file.Upload()

  model.fit(x_train, y_train, nb_epoch=5,validation_data=(x_test, y_test))
  model.save('LSTM4500.h5');
  model_file = drive.CreateFile({'title' : 'LSTM500.h5'})                       
  model_file.SetContentFile('LSTM500.h5')                       
  model_file.Upload()

  model.fit(x_train, y_train, nb_epoch=5,validation_data=(x_test, y_test))
  model.save('LSTM5000.h5'); 
  model_file = drive.CreateFile({'title' : 'LSTM5000.h5'})                       
  model_file.SetContentFile('LSTM5000.h5')                       
  model_file.Upload()
  """
  predictions=model.predict(x_test) 
  mod = gensim.models.Word2Vec.load('word2vec.bin');   
  [mod.most_similar([predictions[10][i]])[0] for i in range(15)]


In [0]:
training()

In [8]:
# -*- coding: utf-8 -*-

import os
from scipy import spatial
import numpy as np
import gensim
import nltk
from keras.models import load_model


import theano
theano.config.optimizer="None"


model=load_model('LSTM500.h5')
mod = gensim.models.Word2Vec.load(os.path.join(word2vec_path,'word2vec.bin'))
while(True):
    x=input("Enter the message:")
    sentend=np.ones((300,),dtype=np.float32) 

    sent=nltk.word_tokenize(x.lower())
    sentvec = [mod[w] for w in sent if w in mod.wv.vocab]

    sentvec[14:]=[]
    sentvec.append(sentend)
    if len(sentvec)<15:
        for i in range(15-len(sentvec)):
          sentvec.append(sentend) 
    sentvec=np.array([sentvec])
    
    predictions = model.predict(sentvec)
    outputlist=[mod.most_similar([predictions[0][i]])[0][0] for i in range(15)]
    output=' '.join(outputlist)
    print(output)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Enter the message:hi


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


know n't n't n't n't but n't n't n't n't n't n't n't n't n't
Enter the message:how are you?
know n't so n't n't but but n't n't n't n't n't n't n't n't
Enter the message:what?
know n't n't n't n't but n't n't n't n't n't n't n't n't n't
Enter the message:what the hell
know n't so n't n't but n't n't n't n't n't n't n't n't n't


KeyboardInterrupt: ignored

**Using Hands-On book approach**

In [29]:

tf.random.set_seed(42)

TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR


NameError: ignored

In [0]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [0]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))
        

In [0]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].repeat().batch(batch_size).prefetch(1)
history = model.fit(train_set, steps_per_epoch=train_size // batch_size, epochs=5)
